# Nelder - Mead with Non-tabu search

## Function and bounds

In [6]:
import copy
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -600, 600
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)
l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]


def f(x):
    N = len(x)
    #Sphere, x_i in [-30,30]
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock, x_i in [-30, -30]
    #Sum = 0
    #for i in range(0, N-1):
    #    Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley, x_i in [-5, -10]
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    
    #Griewank, x_i in [-600,600]
    d = 4000; Sum = 0
    for i in range(0,N): Sum = Sum + (x[i]-100)**2
    Sum = Sum/d; Prod = 1
    for i in range(0,N): Prod = Prod * math.cos((x[i]-100)/math.sqrt(i+1))
    Sum = Sum - Prod +1
    
    #Michalewicz, x_i in [0,math.pi]
    #Sum = 0; m = 10
    #for i in range(0,N): Sum = Sum - math.sin(x[i])*(math.sin(((i+1)*(x[i])**2)/math.pi))**(2*m)
    
    #return value, don't delete
    return Sum

## Algorithm

In [7]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr



def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum



def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False
    
    
    
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw



def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)



def SimplexLocalSearch(x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M and k < 1000:
        #set_trace()
        #if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    #print(f(sBest))
    return sBest, k


def NonTabuSearch(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 2,Beta=-0.5, R = 10, Sigma = 10e-20):
    k = 0
    it = 0
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    x, _k = SimplexLocalSearch(x,Epsilon,Lambda,M-k)
    k = k + _k
    xBest = copy.copy(x)
    
    y = copy.copy(x)
    
    
    while(k < M):
        #set_trace()
        for i in range(1, R):
            for j in range(1,N):
                x[j] = y[j] + Sigma*(u[j] - l[j])
            
            x, _k = SimplexLocalSearch(x,Epsilon,Lambda,M-k)
            k = k + _k
            
            try: xBest
            except NameError: xBest = None
            
            if xBest is None or f(x) < f(xBest):
                print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(x))
                xBest, _k = SimplexLocalSearch(x,EpsilonApostrophe,Lambda,M-k)
                k = k + _k
            
            try: xWorst
            except NameError: xWorst = None
            
            if xWorst is None or f(x) < f(xWorst):
                xWorst = copy.copy(x)
                
        y = copy.copy(xWorst)
        if( xBest is not None and k//1000 > it):
            it = k//1000
            print("iteration:", it * 1000, "| best found value: ", f(xBest))
        
    return xBest

## Main

In [8]:
Lambda = 0.1
M = 100000
Epsilon = 0
#set_trace()
EpsilonApostrophe = 0
x_best = NonTabuSearch(Epsilon,EpsilonApostrophe,Lambda,M)
print(f(x_best))
print(x_best)

***NEW BEST VALUE*** iteration:  1484 | best found value:  2.9431707605458066
***NEW BEST VALUE*** iteration:  6484 | best found value:  0.28436935150982345
iteration: 11000 | best found value:  0.2478735055545065
***NEW BEST VALUE*** iteration:  16599 | best found value:  0.004266555540044847
iteration: 18000 | best found value:  1.212520952420526e-06
***NEW BEST VALUE*** iteration:  18740 | best found value:  3.132882842038498e-11
iteration: 22000 | best found value:  1.887379141862766e-15
***NEW BEST VALUE*** iteration:  23222 | best found value:  0.0


KeyboardInterrupt: 